In [1]:
%load_ext autoreload
%autoreload 2

import os
import torch
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
import matplotlib.pyplot as plt
import numpy as np
from torch.utils.data import DataLoader
from pytorch_lightning import Trainer, seed_everything
from torchvision import transforms
from torchvision.datasets import MNIST
os.chdir("/nfs/students/winter-term-2020/project-4/schwienb/project-4")

In [2]:
from src.data.fashionmnist import FashionMNISTDataModule
from src.models.bg_aug_pixelcnn import BgAugPixelCNN
from src.models.pixelcnn import PixelCNN
from src.experiments.experiment import Experiment
from src.utils.pixelcnn import generate_images, draw_likelihood_plot, positionalencoding2d, randomize_background_normal
from src.utils.ood_test import plot_roc_auc, test_ood, test_ood_coco
import torch.nn.functional as F
from src.models.coco_pixelcnn import COCOPixelCNN
from src.data.coco import COCODataModule
from torchvision import transforms, datasets
from PIL import Image
from torch.utils.data import DataLoader, random_split
import cv2

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
seed_everything(42)


42

In [4]:
experiment_name = 'PixelCNN_BGAUG02_RNBG_RC_COCO_01'

In [5]:
exp = Experiment(experiment_name)

In [6]:
# exp._setup()
exp.load()

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


In [7]:
transform = [
   transforms.Grayscale(num_output_channels=1), transforms.Resize((32,32), interpolation=Image.BICUBIC), 
    transforms.ToTensor()
]
transform = transforms.Compose(transform)

In [8]:
svhn_test = datasets.SVHN('data/', split='test', transform=transform, download=True)

Using downloaded and verified file: data/test_32x32.mat


In [10]:
svhn_testloader = DataLoader(svhn_test, batch_size=1, shuffle=False, num_workers=10, drop_last=True, pin_memory=True)

In [11]:
def test_ood_coco(exp, idd, odd):

    idd_result = exp.trainer.test(exp.model, test_dataloaders=[
                                  idd], verbose=False)
    odd_result = exp.trainer.test(exp.model, test_dataloaders=[
                                  odd], verbose=False)
    idd_results = torch.Tensor(list(map(lambda x: x['test_loss'], idd_result))).numpy()
    odd_results = torch.Tensor(list(map(lambda x: x['test_loss'], odd_result))).numpy()

    targets = np.concatenate(
        (np.zeros(len(idd_results)), np.ones(len(odd_results))))
    results = np.concatenate((idd_results, odd_results))

    return (targets, results)

In [12]:
targets, results = test_ood_coco(exp, exp.dataset.test_dataloader(), svhn_testloader)


Set SLURM handle signals.


Set SLURM handle signals.


ValueError: not enough values to unpack (expected 2, got 1)